# NLP2 project 1
### Mirthe van Diepen and Fabio Curi

First import required libaries.

In [1]:
import numpy as np
import pandas as pd
import tarfile
from collections import *
import matplotlib.pyplot as plt
import random
from decimal import Decimal
from matplotlib.ticker import NullFormatter  # useful for `logit` scale

from scipy import special

from scipy import *
import scipy 
import os


from itertools import accumulate

np.random.seed(19680801)

import copy
import time

## Prepare data

In [2]:
def read_data(tar, names, en_words_u = [], fr_words_u = [], en_words_k = [], fr_words_k = []):
    
    for member in tar.getmembers():
        # check if we want to read this file
        if not member.name in names: continue
        if member.name == names[0]:
            en_tr = tar.extractfile(member).readlines()
        elif member.name == names[1]:
            fr_tr = tar.extractfile(member).readlines()

    # Create dataframe
    data = pd.DataFrame({'en': en_tr,'fr': fr_tr})

    # Get strings and delete \n 
    for i in data.index:
        data.at[i, 'en'] = data.at[i, 'en'].decode("utf-8")
        data.at[i, 'fr'] = data.at[i, 'fr'].decode("utf-8")
        if ' \n' in data.at[i, 'en']:
            data.at[i, 'en'] = data.at[i, 'en'].replace(' \n','')
        if ' \n' in data.at[i, 'fr']:
            data.at[i, 'fr'] = data.at[i, 'fr'].replace(' \n','')

    # create count dictionary 
    dict_e = defaultdict(lambda: 0)
    dict_f = defaultdict(lambda: 0)
    for i in data.index:
        tokens_e = data.at[i, 'en'].split(' ')
        for token in tokens_e:
            dict_e[token] += 1
        tokens_f = data.at[i, 'fr'].split(' ')
        for token in tokens_f:
            dict_f[token] += 1
                
    # Get all bilingual sentences
    sentences_u = []
    sentences_k = []
    for x,y in zip(data['en'], data['fr']):
        sentences_u.append([x.split(),y.split()])
        sentences_k.append([x.split(),y.split()])

    if len(en_words_u) + len(fr_words_u) + len(en_words_k) + len(fr_words_k) == 0:
        for s in sentences_u:
            for i in range(len(s[0])):
                if dict_e[s[0][i]] < 5:
                    # replace rare words by 'unk'
                    s[0][i] = 'unk'
                en_words_u.append(s[0][i])
            for i in range(len(s[1])):
                if dict_f[s[1][i]] < 5:
                    # replace rare words by 'unk'
                    s[1][i] = 'unk'
                fr_words_u.append(s[1][i])
            # add null
            s[0] = ['NULL'] + s[0]
        for s in sentences_k:
            for i in range(len(s[0])):
                en_words_k.append(s[0][i])
            for i in range(len(s[1])):
                fr_words_k.append(s[1][i])
            # add null
            s[0] = ['NULL'] + s[0]
        en_words_u, fr_words_u = sorted(list(set(en_words_u))) , sorted(list(set(fr_words_u)))
        en_words_k, fr_words_k = sorted(list(set(en_words_k))) , sorted(list(set(fr_words_k)))
        print('Size of English dictionary with unknown words:', len(en_words_u))
        print('Size of French dictionary with unknown words:', len(fr_words_u))
        print('Size of English dictionary for k-smoothing:', len(en_words_k))
        print('Size of French dictionary for k-smoothing:', len(fr_words_k))
        return sentences_u, sentences_k, en_words_u, fr_words_u,  en_words_k, fr_words_k
    else:
        for s in sentences_u:
            for i in range(len(s[0])):
                if not s[0][i] in en_words_u:
                    # replace rare words by 'unk'
                    s[0][i] = 'unk'
            for i in range(len(s[1])):
                if not s[1][i] in fr_words_u:
                    # replace rare words by 'unk'
                    s[1][i] = 'unk'
            # add null
            s[0] = ['NULL'] + s[0]
            
        for s in sentences_k:
            # add null
            s[0] = ['NULL'] + s[0]
            
        return data, sentences_u, sentences_k
    

In [3]:
# Read training data
tar = tarfile.open("training.tgz", "r:gz")
names = ["training/hansards.36.2.e", "training/hansards.36.2.f"]
sentences_u, sentences_k, en_words_u, fr_words_u,  en_words_k, fr_words_k = read_data(tar, names)

# Read test data
tar = tarfile.open("testing.tgz", "r:gz")
names = ["testing/test/test.e", "testing/test/test.f"]
test_data, test_sentences_u, test_sentences_k = read_data(tar, names, en_words_u, fr_words_u,  en_words_k, fr_words_k)

# Read validation data
tar = tarfile.open("validation.tgz", "r:gz")
names = ["validation/dev.e", "validation/dev.f", "dev.wa.nonullalign"]
_, validation_sentences_u, validation_sentences_k = read_data(tar, names, en_words_u, fr_words_u,  en_words_k, fr_words_k)

Size of English dictionary with unknown words: 15867
Size of French dictionary with unknown words: 19073
Size of English dictionary for k-smoothing: 36635
Size of French dictionary for k-smoothing: 46421


## AER

In [4]:
# Evolution of alignment error rate (AER) on validation data as a function of the iteration.

"""
This module helps you compute AER for validation set after each iteration of EM.
Check the test function below for an example of how to use these helper functions.

For test results, please use the official AER perl script.
"""

def read_naacl_alignments(path):
    """
    Read NAACL-formatted alignment files.

    :param path: path to file
    :return: a list of pairs [sure set, possible set]
        each entry in the set maps an input position to an output position
        sentences start from 1 and a NULL token is indicated with position 0
    """
    with open(path) as fi:
        ainfo = {}
        for i, line in enumerate(fi.readlines()):
            fields = line.split()
            if not fields:
                continue
            sure = True  # by default we assumed Sure links
            prob = 1.0  # by default we assume prob 1.0
            if len(fields) < 3:
                raise ValueError('Missing required fields in line %d: %s' % (i, line.strip()))
            snt_id, x, y = int(fields[0]), int(fields[1]), int(fields[2])
            if len(fields) == 5:
                sure = fields[3] == 'S'
                prob = float(fields[4])
            if len(fields) == 4:
                if fields[3] in {'S', 'P'}:
                    sure = fields[3] == 'S'
                else:
                    prob = float(fields[3])
            snt_info = ainfo.get(snt_id, None)
            if snt_info is None:
                snt_info = [set(), set()]  # S and P sets
                ainfo[snt_id] = snt_info
            if sure:  # Note that S links are also P links: http://dl.acm.org/citation.cfm?id=992810
                snt_info[0].add((x, y))
                snt_info[1].add((x, y))
            else:
                snt_info[1].add((x, y))
    return tuple(v for k, v in sorted(ainfo.items(), key=lambda pair: pair[0]))


class AERSufficientStatistics:
    """
    Object used to compute AER for a corpus.
    """

    def __init__(self):
        self.a_and_s = 0
        self.a_and_p = 0
        self.a = 0
        self.s = 0

    def __str__(self):
        return '%s/%s/%s/%s %s' % (self.a_and_s, self.a_and_p, self.a, self.s, self.aer())

    def update(self, sure, probable, predicted):
        """
        Update AER sufficient statistics for a set of predicted links given goldstandard information.

        :param sure: set of sure links 
            a links is a tuple of 1-based positions (from, to) where 0 is reserved for NULL
        :param probable: set of probable links (must incude sure links)
        :param predicted: set of predicted links
        """
        self.a_and_s += len(predicted & sure)
        self.a_and_p += len(predicted & probable)
        self.a += len(predicted)
        self.s += len(sure)

    def aer(self):
        """Return alignment error rate: 1 - (|A & S| + |A & P|)/(|A| + |S|)"""
        return 1 - (self.a_and_s + self.a_and_p) / (self.a + self.s)


def test(path, predictions):
    
    # read in gold alignments
    gold_sets = read_naacl_alignments(path)

    # compute AER:
    print(type(gold_sets))
    print(type(predictions))
    # first we get an object that manages sufficient statistics 
    metric = AERSufficientStatistics()
    # then we iterate over the corpus 
    for gold, pred in zip(gold_sets, predictions):
        metric.update(sure=gold[0], probable=gold[1], predicted=pred)
    
    return metric.aer()

def all_indices(value, qlist):
    indices = []
    idx = -1
    while True:
        try:
            idx = qlist.index(value, idx+1)
            indices.append(idx)
        except ValueError:
            break
    return indices[0]

def align_IBM1(s, t):
    
    alignments=[]
    l = len(s[0])
    m = len(s[1])
    for i in range(l):
        if i==0: continue
        vals=[]
        for j in range(m):
            vals.append(t[s[0][i]][s[1][j]])
        max_idx = vals.index(max(vals))
        alignments.append((i, max_idx+1)) # try +1

    return set(alignments)

def align_IBM2(s, pi_t, pi_A):
    
    alignments=[]
    l = len(s[0])
    m = len(s[1])
    for i in range(l):
        if i==0: continue
        vals=[]
        for j in range(m):
            vals.append(pi_A[l][m][j][i] * pi_t[s[0][i]][s[1][j]])
        max_idx = vals.index(max(vals))
        alignments.append((i, max_idx+1))

    return set(alignments)

## Variational inference for Bayesian IBM model 1

In [5]:
def ELBO(psi, alpha, lamb, theta, epsilon, sentences):
    
    Vf = len(fr_words_u)
    Ve = len(en_words_u)
    part1 = 0
    part2 = 0
    
    lamb_fr = defaultdict(lambda: 0.)
    for k, s in enumerate(sentences):
        e, f = s[0], s[1]
        l = len(s[0])
        m = len(s[1])
        for j in range(m):
            Q = epsilon
            for i in range(l):
                e, f = s[0][i], s[1][j]
                Q *= psi[k][e][f]**i
                part2 += theta[e][f]*(alpha-lamb[e][f]) + log(scipy.stats.gamma(lamb[e][f]))
                lamb_fr[e] += lamb[e][f]
            part1 += log(Q)
    for e in en_words_u:
        part2 -= log(scipy.stats.gamma(lamb_fr[e]))
    part2 += Ve * log(scipy.stats.gamma(Vf * alpha))
    
    return part1 + part2

In [6]:
# alpha is not depending on f

als = []

def VIB(sentences, val_sentences, alpha=0.1, iterations=10):
    
    lambd = defaultdict(lambda: defaultdict(lambda: 0.5))
    theta = defaultdict(lambda: defaultdict(lambda: 0.5))
    
    # initialise counts
    lambda_total = defaultdict(lambda: 0.)
    theta_total = defaultdict(lambda: defaultdict(lambda: 0.))
    count_e2f = defaultdict(lambda: defaultdict(lambda: 0.))
    
    perplexity_vals, AER_vals, house_maison, house_demain = [], [], [], []
    
    start = time.time()
    for s in sentences:
        for e in s[0]:
            for f in s[1]:
                lambda_total[e] += lambd[e][f]
                count_e2f[e][f] += 1
    end = time.time()
    print('first loop: {}'.format(end-start))
    
    for p in range(iterations):
        
        print("Iteration {}".format(p))     
        
        # E-step
        start = time.time()
        for k, s in enumerate(sentences):
            for e in s[0]:
                theta_total[e] = 0
                for f in s[1]:
                    theta[e][f] = np.exp(scipy.special.digamma(lambd[e][f])- scipy.special.digamma(lambda_total[e]))
                    theta_total[k][f] += theta[e][f]
        end = time.time()
        print('second loop: {}'.format(end-start))
        
        # M-step
        start = time.time()
        for k, s in enumerate(sentences):
            for e in s[0]:
                lambda_total[e] = 0 # fout
                for f in s[1]:
                    expectation = count_e2f[e][f] * theta[e][f] / theta_total[k][f]
                    lambd[e][f] = alpha + expectation
                    lambda_total[e] += lambd[e][f]
        end = time.time()
        print('third loop: {}'.format(end-start))
        
        # Store some results for plot
        house_maison.append(lambd['house']['maison'])
        house_demain.append(lambd['house']['demain'])
                    
        
        # Simulation of a close-to Viterbi algorithm to calculate the AER for each iteration
        alignments = []
        for s in val_sentences:
            alignments.append(align_IBM1(s,lambd))
        als.append(alignments)
                    
    return lambd, perplexity_vals, als, house_maison, house_demain

In [7]:
lambd_bayes, perplexity_vals_bayes, als_bayes, hm_bayes, hd_bayes = VIB(sentences_u, validation_sentences_u, 0.001, 1)

first loop: 79.24879002571106
Iteration 0
second loop: 546.9475557804108
third loop: 181.96502995491028


In [9]:
def create_naacl_IBM1(t, test_sentences, path):
    f = open(path, "w+")
    N = len(test_sentences)
    for n, s in enumerate(test_sentences):
        l = len(s[0])
        m = len(s[1])
        for i in range(l):
            if i==0: continue
            vals=[]
            for j in range(m):
                vals.append(t[s[0][i]][s[1][j]])
            max_idx = vals.index(max(vals))
            if n==N-1 and i==l-1 and j==m-1:
                f.write("{} {} {} S".format(n+1, i, max_idx+1))
            else:
                f.write("{} {} {} S\n".format(n+1, i, max_idx+1))
    f.close()
    
create_naacl_IBM1(lambd_bayes, validation_sentences_u, "ibm1.vb.naacl")